# DEMO RAG GRUPO PLANETA

# Preparations

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4 pypdf docx2txt faiss-cpu fasttext-langdetect streamlit

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = "" #getpass.getpass()

# import dotenv

# dotenv.load_dotenv()

In [95]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader, DirectoryLoader #, WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
from time import sleep
from ftlangdetect import detect

### Data

### Load Documents

In [40]:
loader = DirectoryLoader('./documents_planeta/', glob="*.docx", loader_cls=Docx2txtLoader)

data = loader.load()
data

[Document(page_content='ACCOUNT\n\nEn la entidad “Account” en la pestaña “Summary” en la sección “Account Address” aparecerán cinco campos importantes. Los campos son:\n\nCountry, que es de tipo lookup\n\nState, que es de tipo lookup\n\nState Text, que es de tipo text.\n\nCity, que es de tipo lookup.\n\nCity Text, que es de tipo text.\n\nDependiendo de si el país esta normalizado o no la entidad “Account” recibirá unos valores u otros de la entidad “Lead”. En concreto será:\n\nSi el país esta normalizado. Los campos “Country”, “State”, “City” de la entidad “Account” heredan de los campos “Country”, “State”, y “City” de la entidad “Lead”. El campo “State Text” tendrá el mismo valor que el campo “State”, y el campo “City Text” tendrán el mismo valor que el campo “City”.\n\nSi el país no está normalizado. Los campos “Country”, “State Text”, “City Text” de la entidad “Account” heredan de los campos “Country”, “State Text”, y “City Text” de la entidad “Lead”.\n\n\n\nIlustración 4. Campos "C

### Split Documents

In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents=data)
splits

[Document(page_content='ACCOUNT\n\nEn la entidad “Account” en la pestaña “Summary” en la sección “Account Address” aparecerán cinco campos importantes. Los campos son:\n\nCountry, que es de tipo lookup\n\nState, que es de tipo lookup\n\nState Text, que es de tipo text.\n\nCity, que es de tipo lookup.\n\nCity Text, que es de tipo text.\n\nDependiendo de si el país esta normalizado o no la entidad “Account” recibirá unos valores u otros de la entidad “Lead”. En concreto será:\n\nSi el país esta normalizado. Los campos “Country”, “State”, “City” de la entidad “Account” heredan de los campos “Country”, “State”, y “City” de la entidad “Lead”. El campo “State Text” tendrá el mismo valor que el campo “State”, y el campo “City Text” tendrán el mismo valor que el campo “City”.\n\nSi el país no está normalizado. Los campos “Country”, “State Text”, “City Text” de la entidad “Account” heredan de los campos “Country”, “State Text”, y “City Text” de la entidad “Lead”.', metadata={'source': 'document

### Indexing

In [42]:
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

### Retrieval and Generation: Retrieve

In [43]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

retrieved_docs = retriever.invoke("How can I document the ticket?")

for retrieved_doc in retrieved_docs:
    print()
    print(retrieved_doc)


page_content='Mínimo 2 veces al día (a primera hora de la mañana y primera hora de la tarde)\n\n\n\n\n\nC13\n\nSeguimiento de tickets escalados a N2\n\n\xa0\n\nLos tickets escalados a N2, que tienen un BASM/BTA/BTED/BTPW asociado sobre el que trabaja el Equipo de N2, deben ser revisados y actualizados periódicamente, anotando los avances que se hayan podido producir. Ello permitirá dar visibilidad a los peticionarios del punto en el que está su solicitud.\n\nSemanalmente\n\n\n\nComunicaciones\n\nC14\n\nMedios de contacto\n\n\xa0\n\nLas incidencias llegarán mediante Jira, siendo este soporte el medio de comunicación con toda la organización, sobre el caso tratado.\n\n\xa0\n\nExcepcionalmente y a criterio del Técnico, para agilizar la resolución, se podrán utilizar otros medios de contacto, quedando fielmente reflejado tal como se indica en el punto Operativa à Documentar el Ticket.\n\n\xa0\n\nLos comentarios se añadirán al ticket personalizando el usuario con @Usuario para que reciba u

### Retrieval and Generation: Generation

In [21]:
# prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [105]:
template = """You are an assistant for providing guide and instructions for two tasks: tickets incidences management and user manual for a web application.
Use the following pieces of context to answer the question at the end.
If the question can not be answered using the information of the context, just say that you don't know, don't try to make up an answer.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

Context: {context}

Question: {question}

Answer in the following language: {language}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [106]:
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     { "context": retriever | format_docs,
#       "question": RunnablePassthrough() ,
#       "language": itemgetter("language"),
#     }
#     | custom_rag_prompt
#     | llm
#     | StrOutputParser()
# )
rag_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [70]:
# rag_chain.invoke("How can I document the ticket?")
rag_chain.invoke({"question": "How can I document the ticket?", "language": "spanish"})

'Se debe informar cualquier contacto/reunión/llamada, etc. que se haya mantenido con el peticionario, para que conste todo el histórico de las acciones realizadas. Además, se debe informar la resolución en el campo comentario en base a los formatos estándares definidos. Gracias por preguntar!'

In [119]:
question = "How can I document the ticket?"
question = "Cómo documento un ticket?"
question = "Qué valor se debe introducir en los campos de la entidad 'Account' cuando el país está normalizado?"
question = "Which value I should use to fill the fields of the 'Account' entity when the country is normalized?"
# question = "Cómo puedo borrar un ticket?"
question = "De qué color son los plátanos?"

for chunk in rag_chain.stream({"question": question, "language": detect(text=question, low_memory=False)["lang"]}):
    print(chunk, end="", flush=True)
    sleep(0.05)

No se sabe de qué color son los plátanos según la información proporcionada. Gracias por preguntar!

In [118]:
retrieved_docs = retriever.invoke(question)

for index, retrieved_doc in enumerate(retrieved_docs):
    print("\r\n------------------------------------------")
    print(f"Fragmento de documento: {index}")
    print(retrieved_doc.page_content)    


------------------------------------------
Fragmento de documento: 0
Si el país no está normalizado. Los campos “Country”, “State Text”, “City Text” de la entidad “Account” heredan de los campos “Country”, “State Text”, y “City Text” de la entidad “Lead”.



Ilustración 4. Campos "Country", "State", "State Text", "City", y "City Text" en la seccion "Account Address" en la pestaña "General".

El gestor podrá modificar los valores de los campos “Country”, “State”, “State Text”, “City”, y “City Text” teniendo en cuanta las siguientes consideraciones:

Si el campo “Country” es un país normalizado:

Si el valor del campo “Normalizado” en la entidad “Country” es igual a “Yes”, los campos “State”, y “City” serán visibles y obligatorios; mientras que los campos “State Text”, y “City Text” serán visibles, pero aparecerán bloqueados.

El campo “State Text” recibirá el valor que tenga el campo “Text”.

El campo “City Text” recibirá el valor que tenga el campo “City”.

---------------------------